# GRAPH FEATURES

Here we will compute for the three graphs (full graph, test graph, train graph), some features:

- **Betweeness centrality** of all the nodes

We  use the python library `ensmallen` which is available when downloading the library `grape`. [GraPE](https://github.com/AnacletoLAB/grape) (Graph Processing and Embedding) is a fast graph processing and embedding library, designed to scale with big graphs and to run on both off-the-shelf laptop and desktop computers and High Performance Computing clusters of workstations.

In [1]:
import os
import pickle
import networkx as nx
from ensmallen import Graph

import paths

## 1. Load the graphs

In [2]:
# Complete graph
full_graph = Graph.from_csv(
    edge_path=paths.FULL_GRAPH_EDGELIST_PATH,
    edge_list_separator=",",
    edge_list_header=False,
    sources_column_number=0,
    destinations_column_number=1,
    edge_list_numeric_node_ids=True,
    directed=False,
)

# Train graph
train_graph = Graph.from_csv(
    edge_path=paths.TRAIN_EDGELIST_PATH,
    edge_list_separator=",",
    edge_list_header=False,
    sources_column_number=0,
    destinations_column_number=1,
    edge_list_numeric_node_ids=True,
    directed=False,
)

# Test graph
test_graph = Graph.from_csv(
    edge_path=paths.TEST_EDGELIST_PATH,
    edge_list_separator=",",
    edge_list_header=False,
    sources_column_number=0,
    destinations_column_number=1,
    edge_list_numeric_node_ids=True,
    directed=False,
)

## 2. Compute Betweeness centrality

In [15]:
# Full graph
if not os.path.isfile(paths.FULL_GRAPH_BET_CENT_PATH):
    full_graph_bet_cen = full_graph.get_betweenness_centrality(verbose=True)
    with open(paths.FULL_GRAPH_BET_CENT_PATH, 'wb') as f:
        pickle.dump(full_graph_bet_cen, f)
else:
    print("The betweeness centrality for the full graph was already computed !")

The betweeness centrality for the full graph was already computed !


In [ ]:
# train graph
if not os.path.isfile(paths.TRAIN_GRAPH_BET_CENT_PATH):
    train_graph_bet_cen = train_graph.get_betweenness_centrality(verbose=True)
    with open(paths.TRAIN_GRAPH_BET_CENT_PATH, 'wb') as f:
        pickle.dump(train_graph_bet_cen, f)
else:
    print("The betweeness centrality for the train graph was already computed !")

In [ ]:
# Test graph
if not os.path.isfile(paths.TEST_GRAPH_BET_CENT_PATH):
    test_graph_bet_cen = test_graph.get_betweenness_centrality(verbose=True)
    with open(paths.TEST_GRAPH_BET_CENT_PATH, 'wb') as f:
        pickle.dump(test_graph_bet_cen, f)
else:
    print("The betweeness centrality for the test graph was already computed !")

## 3. Compute Closeness centrality

In [ ]:
# Full graph
if not os.path.isfile(paths.FULL_GRAPH_BET_CENT_PATH):
    full_graph_clo_cen = full_graph.get_closeness_centrality(verbose=True)
    with open(paths.FULL_GRAPH_BET_CENT_PATH, 'wb') as f:
        pickle.dump(full_graph_clo_cen, f)
else:
    print("The closeness centrality for the full graph was already computed !")

The betweeness centrality for the full graph was already computed !
